In [1]:
import torch
import random

from util import TokenizerUtil

tokenizer = TokenizerUtil()

input_ids, attention_mask = tokenizer.encode('how are you', max_length=4)

input_ids, attention_mask, tokenizer.decode(input_ids)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(tensor([128000,   5269,    527, 128001]),
 tensor([1, 1, 1, 1]),
 '<|begin_of_text|>how are<|end_of_text|>')

In [2]:
from datasets import load_dataset
from transformers import default_data_collator

dataset = load_dataset('json', data_files='dataset/train.json', split='train')

#4,2,4切分,取第0部分
dataset = dataset.select(range(30000))


def f(data):
    #随机生成两种回答
    if random.random() > 0.5:
        data['chosen'] = data['chosen'].swapcase()
    data = data['prompt'] + data['chosen']

    input_ids, attention_mask = tokenizer.encode(data)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': input_ids.clone()
    }


dataset = dataset.map(f, remove_columns=dataset.column_names)

loader = torch.utils.data.DataLoader(dataset,
                                     collate_fn=default_data_collator,
                                     batch_size=4,
                                     shuffle=True,
                                     drop_last=True)

len(loader), next(iter(loader)).keys()

(7500, dict_keys(['input_ids', 'attention_mask', 'labels']))

In [3]:
%run 1.model.ipynb
from accelerate import Accelerator

model_actor = LlamaForCausalLM()

model_actor.train()

optimizer = torch.optim.Adam(model_actor.parameters(), lr=1e-4)

accelerator = Accelerator(mixed_precision='fp16')

loader, model_actor, optimizer = accelerator.prepare(loader, model_actor,
                                                     optimizer)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [4]:
for i, data in enumerate(loader):
    loss, logits = model_actor(**data)
    accelerator.backward(loss)
    accelerator.clip_grad_norm_(model_actor.parameters(), 1.0)
    optimizer.step()
    optimizer.zero_grad()

    if (i + 1) % 100 == 0:
        print(i, len(loader), loss.item())
        logits = logits[0].argmax(1)
        print(tokenizer.decode(logits))

torch.save(model_actor.to('cpu'), 'model/actor')

99 7500 2.580916404724121
Human: context= CREATE TABLE table_name1 ( ( (1 (date VARCHAR VARCHAR, score VARCHAR VARCHAR VARCHAR) question= What had of player is201? the?5 the where:SELECT MIN = = table_name1 FROM"_1 WHERE date"" = "201<|end_of_text|>
199 7500 2.2653822898864746
Human: context= CREATE TABLE table_name_1 (points VARCHAR, date_par VARCHAR) name VARCHAR) question= What is the lowest of the? has a??? of Assistant:select count from TABLE_NAME_1 where NAMEURN = "1 WHERE<|end_of_text|>
299 7500 1.2085587978363037
Human: context= CREATE TABLE table_name_17 (date VARCHARub_team, name VARCHAR) question= Which date has has a score? 2? Assistant:select date fromub_team table_name_17 WHERE date > "2<|end_of_text|>
399 7500 1.7112624645233154
Human: context= CREATE TABLE table_name_61 (time VARCHAR VARCHARpercentage, year VARCHAR) question= What many rounds has a of than 200? Assistant:SELECT min()) frompercentage from SCORE from TABLE_NAME_1 where VINS < 200<|end_of_text|>
499 7500 1

2299 7500 2.171806573867798
Human: context= CREATE TABLE table_name_94 (year VARCHARade VARCHAR, name VARCHAR) name VARCHAR VARCHARade VARCHAR) question= What is the final thatris when the frequency of thatris that a frequency of ??le??ret????�?�?? Assistant:select CH fromADE from TABLE_NAME_94 where CH qualify = = =ADE = "B"IE" and CAMES = "<|end_of_text|>
2399 7500 0.9664218425750732
Human: context= CREATE TABLE table (name_id VARCHAR, staff VARCHAR VARCHAR VARCHAR VARCHAR); question= Show are the names of the game common year by Assistant:SELECT T_id FROM staff WHERE BY date_of_idaced WHERE<|end_of_text|>
2499 7500 1.1220698356628418
Human: context= CREATE TABLE table_name_93 (score VARCHAR, date VARCHAR) tournament VARCHAR) question= Which is the surface for the, with with with a Score  of of Assistant:SELECT surface FROM table_name_94 WHERE tournament = "k"" AND tournament = "kinn"<|end_of_text|>
2599 7500 1.056701421737671
Human: context= CREATE TABLE table_name_98 (score INTEGER

4799 7500 1.487322449684143
Human: context= CREATE TABLE table_name_55 (playeroists_order VARCHAR, title VARCHAR) question= What railway has title is a I title of????? Assistant:SELECT platformatial_title FROM table_name_77 WHERE platform = "kitt"""""<|end_of_text|>
4899 7500 1.3078619241714478
Human: context= CREATE TABLE table (name_code, city_codecode VARCHAR); question= Show the name of the of '100. Assistant:select city FROM Student ORDER NOT_code = = '100<|end_of_text|>
4999 7500 1.067262887954712
Human: context= CREATE TABLE table_name_20 (year INTEGER, question= What is the rank11 value a  of than 1. a 10rer?? Assistant:select 2010 FROM table_name_72 WHERE  > 1 AND 2011 = "1"""<|end_of_text|>
5099 7500 0.7336298227310181
Human: context= CREATE TABLE table_name_92 (dateponent VARCHAR_the_final VARCHAR, date VARCHAR) question= Who was the opponent in the final on December2/ 2009? Assistant:SELECT OPPONENT_IN_THE_FINAL from TABLE_NAME_43 where DATE = "24/VEMBER 2008"<|end_of_text|

7399 7500 0.5813372135162354
Human: context= CREATE TABLE table_name_67 (date INTEGER, rank VARCHAR, question= What is is season 200? Assistant:SELECT rank FROM table_name_28 WHERE season = "1<|end_of_text|>
7499 7500 0.8609915971755981
Human: context= CREATE TABLE table_name_47 (opze INTEGER, silver VARCHAR) question= What is the average of bronze medals the total is more than 0? Assistant:SELECT SUM(bronze) FROM table_name_58 WHERE total < 1<|end_of_text|>
